# Lab 1 - SQL, Sakila EDA 
## Task 0

#### Data Ingestion (sqlite to duckdb)

In [10]:
# ================================
# Task 0 – Data Ingestion (DuckDB)
# ================================
# This cell loads the Sakila database into DuckDB.
# It should be the FIRST code cell in your notebook.

import duckdb
from pathlib import Path

# Path where DuckDB database will be created
duckdb_path = "data/sakila.duckdb"

# Delete existing DuckDB file (if it exists) to start fresh
Path(duckdb_path).unlink(missing_ok=True)

# Connect to DuckDB and run the Sakila load script
with duckdb.connect(duckdb_path) as conn:
    
    # Read and execute the SQL file that loads Sakila tables
    with open("sql/load_sakila.sql", "r") as f:
        conn.sql(f.read())
    
    # Check that tables are created successfully
    tables = conn.sql("SHOW TABLES;").df()
    tables


In [ ]:
# This cell loads the Sakila database into duckdb.
import duckdb
from pathlib import Path

# Path where duckdb database will be created
duckdb_path = "data/sakila.duckdb"

# Delete existing duckdb file if it exists to start fresh
Path(duckdb_path).unlink(missing_ok=True)


# Connect to duckdb and run the sakila load script
with duckdb.connect(duckdb_path) as conn, open("sql/load_sakila.sql") as ingest_script:
    conn.sql(ingest_script.read())

    # Show database schema (list of tables & columns)
    description = conn.sql("DESC;").df()
    # Load the entire film table into a pandas dataframe
    films = conn.sql("FROM film;").df()

# Show the first 3-rows to verify the data was loaded correctly
films.head(3)

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,<NA>,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2021-03-06 15:52:00
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,<NA>,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2021-03-06 15:52:00
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,<NA>,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2021-03-06 15:52:00


In [7]:
import os
os.path.isfile("sakila.db")  # should return True

False

In [8]:
# Import Liabraries...
# sqlite to read database
# duckdb helps to create a duckdb database
# pandas use to load and move the data

# import sqlite3
import duckdb
import pandas  as pd
# import matplotlib.pyplot as plt 

In [9]:
# Connect to SQLite Sakila database (source data)
sqlite_conn = sqlite3.connect("sakila.db")  # make sure path is correct
print("Connected to sqlite")

NameError: name 'sqlite3' is not defined

In [ ]:
# Get real sakila tables from sqlite (ignore sqlite_master)
tables = pd.read_sql("""
    SELECT name 
    FROM sqlite_master 
    WHERE type='table' 
    AND name NOT LIKE 'sqlite_%'
    """,
    sqlite_conn
)

# print("SQLite tables:")
tables      # display the list of tables..each row under name is table name(film,actor,customer, rental, payment etc...)

In [ ]:
# # ---Confirmation that sqlite has film table---
# sqlite_tables = pd.read_sql("""
# SELECT name
# FROM sqlite_master
# WHERE type='table'
# """, sqlite_conn)

# print("SQLite tables:")
# print(sqlite_tables)

### Connect to duckdb database

In [ ]:
# Connect to duckdb (creates file if it does not exist)
duckdb_conn = duckdb.connect("sakila.duckdb")

print("Connected to duckdb database")

In [ ]:
# Copy tables...from sqlite to duckdb
for table_name in tables['name']:
    df = pd.read_sql(f"SELECT * FROM {table_name}", sqlite_conn)
    duckdb_conn.execute(
        f"CREATE OR REPLACE TABLE {table_name} AS SELECT * FROM df"
    )


In [ ]:
# # --- For this particular cell took help from CGPT---

# # Drop everything in duckdb
# duckdb_conn.execute(""" 
# PRAGMA show_tables;
# """)

In [ ]:
# Verify tables in duckdb
print("DuckDB tables:")
duckdb_conn.execute("SHOW TABLES").df()

##### Close sqlite database connection (duckdb stays open)

In [ ]:
sqlite_conn.close()

print("Connections closed")

## Task  1 -     (EDA)Exploratory Data Analysis
- Q: a-f using duckdb + pandas

In [20]:
# use duckdb for analysis

import duckdb
db = duckdb.connect("sakila.duckdb") 

In [ ]:
# Check tables
db.execute("SHOW TABLES").df()

#### a) Which movies are longer than 3hours (180 min)?
- Show:  title & length

In [ ]:
# Select movies longer than 180 minutes
query = """ 
SELECT title, length
FROM film           
WHERE length > 180
ORDER BY length DESC 
"""

db.execute(query).df()

# movies_longer_than_3h = db.execute(query).df()
# movies_longer_than_3h 

#### b) Movies with the word "love" in the title

In [ ]:
# Find movies that contain 'love' in the title
query = """ 
SELECT title, rating, length, description
FROM film
WHERE LOWER(title) LIKE '%love%'
"""

love_movies = con.execute(query).df()
love_movies

#### c) Movies length statistics
We calculate:
- shortest
- average
- median
- longest

In [ ]:
query = """ 
SELECT
    MIN(length) AS shortest,
    AVG(length) AS average,
    MEDIAN(length) AS median,
    MAX(length) AS longest
FROM film
"""

movie_stats = con.execute(query).df()
movie_stats

#### d) 10 most expensive movies to rent per day

In [ ]:
cost per day = rental_rate / rental_duration

query = """ 
SELECT title,
       rental_rate,
       rental_duration,
       rental_rate / rental_duration AS cost_per_day
FROM film
ORDER BY cost_per_day DESC
LIMIT 10
"""

expensive_movies = con.execute(query).df()
expensive_movies

#### e) Top 10 actors with the number of movies they have played in

In [ ]:
query = """ 
SELECT
    a.first_name || '' || a.last_name AS actor_name,
    COUNT(fa.film_id) AS movie_count
FROM actor a
JOIN film_actor fa ON a.actor_id = fa.actor_id
GROUP BY actor_name
ORDER BY movie_count DESC
LIMIT 10
"""

top_actors = con.execute(query).df()
top_actors 

#### f) My own 3 simple questions


Q_1 : Which categories have most movies?

In [6]:
query = """ 
SELECT c.name AS category, COUNT(*) AS movies
FROM category c
JOIN film_category fc ON c.category_id = fc.category_id
GROUP BY category
ORDER BY movies DESC
"""

category_movies = con.execute(query).df()
category_movies 

CatalogException: Catalog Error: Table with name category does not exist!
Did you mean "pg_attrdef"?

LINE 3: FROM category c
             ^

Q_2: Average rental rate per category

In [ ]:
query = """ 
SELECT c.name AS category, AVG(f.rental_rate) AS avg_rate
FROM category c
JOIN film_category fc ON c.category_id = fc.category_id
JOIN film f ON fc.film_id = f.film_id
GROUP BY category
"""

avg_rates = con.execute(query).df()
avg_rates

Q_3: Top 5 customers by number of rentals

In [ ]:
query = """ 
SELECT c.first_name || '' || c.last_name AS customer,
       COUNT(r.rental_id) AS rentals
FROM customer c
JOIN rental r ON c.customer_id = r.customer_id
GROUP BY customer
ORDER BY rentals DESC
LIMIT 5
"""

top_renters = con.execute(query).df()
top_renters

## Task 2 (Graphs) - Visualizations

#### a) Top 5 customers by total spend

In [7]:
query = """ 
SELECT c.first_name || '' || c.last_name AS customer,
       SUM(p.amount) AS total_spend
FROM customer c
JOIN payment p ON c.customer_id = p.customer_id
GROUP BY customer
ORDER BY total_spend DESC
LIMIT 5
"""

top_spend = con.execute(query).df()

# Bar chart
plt.figure()
plt.bar(top_spend['customer'], top_spend['total_spend'])
plt.title('Top 5 Customers by Total Spend')
plt.xlabel('Customer')
plt.ylabel('Total Spend')
plt.show()

CatalogException: Catalog Error: Table with name customer does not exist!
Did you mean "pg_constraint or sqlite_master"?

LINE 4: FROM customer c
             ^

#### b) Revenue per film category

In [8]:
query = """ 
SELECT c.name AS category,
       SUM(p.amount) AS revenue
FROM category c
JOIN film_category fc ON c.category_id = fc.category_id
JOIN inventory i ON fc.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
JOIN payment p ON r.rental_id = p.rental_id
GROUP BY revenue DESC
"""

revenue_category = con.execute(query).df()

plt.figure()
plt.bar(revenue_category['category'], revenue_category['revenue'])
plt.title('Revenue per Film Category')
plt.xlabel('Category')
plt.ylabel('Revenue')
plt.xticks(rotation=45)
plt.show()

ParserException: Parser Error: syntax error at or near "DESC"

LINE 9: GROUP BY revenue DESC
                         ^